In [56]:
#This script is using to scrape all posts from 2022
from bs4 import BeautifulSoup
import requests
import re
from lxml import html
from datetime import datetime
from itertools import chain
import pandas as pd
url='https://ficoforums.myfico.com/t5/Personal-Finance/bd-p/Retirement'


In [29]:

def get_num_pages(url):
    last_page=''
    content=requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")
    pages=soup.find_all('li', {'class':'lia-component-pagesnumbered'})
    for page in pages[:1]:
        last_page=soup.find('li', {'class':re.compile('lia-paging-page-last')}).text
    pages=list(range(2,int(last_page)+1))
    return(pages)
        
    

##pager > ul > li.lia-component-pagesnumbered

In [3]:
def get_post_date(message_cell):
    date_cell=message_cell.find_all('span', {'class':re.compile('lia-component-common-widget-date')})
    post_date_cell=date_cell[0]
    post_date=post_date_cell.find('span', {'class':'local-date'}).text.strip()
    post_date=post_date.encode('ascii','ignore')
    post_date=post_date.decode()
    post_date=datetime.strptime(post_date, '%m-%d-%Y')
    return post_date.date()

    




In [17]:
def get_post_author(message_cell):
    author=''
    author_name_cell=message_cell.find_all('span', {'class':'login-bold'})
    for a in author_name_cell[:1]:
        author=a.text.strip()
    return author

In [18]:
def get_post_title(message_cell):
    post_title=message_cell.find('h2',{'class':'message-subject'}).text.replace('\n',' ').strip()
    return post_title

In [22]:
def get_post_link(message_cell):
    post_link=message_cell.find('a', {'class':re.compile('page-link')})['href']
    return f'https://ficoforums.myfico.com{post_link}'

In [63]:
def get_reply_count(message_cell):
    replies_cell=message_cell.find('div', {'class':'lia-component-messages-column-message-replies-count'})
    replies=replies_cell.find('span', {'class':'lia-message-stats-count'}).text.replace(',','').strip()
    return int(replies)
    




In [30]:
def create_url_pages(url):
    pages_url=[url]
    pages_num=get_num_pages(url)
    for p in pages_num:
        p_url=f'{url}/page/{str(p)}'
        pages_url.append(p_url)
    return(pages_url)

In [65]:

def get_message(url):
    posts_2022=[]
    date_from=datetime(2021,12,31).date()
    content=requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")
    topics=soup.find_all('tr', {'class':re.compile('lia-list-row')})
    for topic in topics:
        date=get_post_date(topic)
        author=get_post_author(topic)
        title=get_post_title(topic)
        link=get_post_link(topic)
        replies=get_reply_count(topic)
        message_id=int(link.split('/')[-1])
        
        if (date<date_from) or (replies==0):
            continue
        else:
            posts_2022.append((message_id, date, title, author, replies, link))
    return posts_2022
        



    
    

In [66]:
from tqdm import tqdm
def get_all_messages(url):
    pages=create_url_pages(url)
    posts_2022_all=[]
    for u in tqdm(pages):
        posts_2022_all.append(get_message(u))
    posts_df=pd.DataFrame(list(chain(*posts_2022_all)), columns=['post_id','post_date','post_title','post_author','post_replies', 'post_link'])
    return posts_df
    
    

In [67]:
posts_df=get_all_messages(url)
posts_df

100%|██████████| 196/196 [04:01<00:00,  1.23s/it]


,post_id,post_date,post_title,post_author,post_replies,post_link
0,6619712,2022-11-30,Garden Club - December 2022,UncleB,535,https://ficoforums.myfico.com/t5/Personal-Fina...
1,6624791,2022-12-16,The 42 billion dollar question,AndySoCal,3,https://ficoforums.myfico.com/t5/Personal-Fina...
2,6626687,2022-12-23,Navy Federal Internal Score,tebtengri,5,https://ficoforums.myfico.com/t5/Personal-Fina...
3,6604324,2022-10-16,high yield savings accounts,acreditdummy123,20,https://ficoforums.myfico.com/t5/Personal-Fina...
4,6612558,2022-11-07,Does the Fold debit card check ChexSystems?,TyrannicalDuncery,6,https://ficoforums.myfico.com/t5/Personal-Fina...
...,...,...,...,...,...,...
482,6468010,2022-01-04,New blog article: How Minimalism Can Help You ...,Elizabeth_FICO,2,https://ficoforums.myfico.com/t5/Personal-Fina...
483,6466879,2022-01-01,Tucson Federal Credit Union HP for regular sav...,SouthJamaica,2,https://ficoforums.myfico.com/t5/Personal-Fina...
484,6466656,2022-01-01,PENFED Login,outofcredit,3,https://ficoforums.myfico.com/t5/Personal-Fina...
485,6381660,2022-04-11,Charles Schwab Investor Checking,UncleB,14,https://ficoforums.myfico.com/t5/Personal-Fina...


In [68]:
posts_df.to_csv('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/posts_to_scrape.csv', index=False)